# Import Pacchetti

In [1]:
import pandas as pd
import matplotlib as plm
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors as mcolors
import seaborn as sns
import numpy as np
from numpy import median
from seaborn import JointGrid

# Cleaning e manipolazione Dataset

## Import + clenaning

In [18]:
df = pd.read_csv('compitino_dataviz_2021_2022.csv')

In [19]:
physical = df[df['nome_modulistica'] == 'SF12 PhysicalScore'].reset_index()
physical = physical.rename(columns = {'score_preop':'physical_score_preop', 'score_postop':'physical_score_postop'})

In [20]:
mental = df[df['nome_modulistica'] == 'SF12 MentalScore'].reset_index()
mental = mental.rename(columns = {'score_preop':'mental_score_preop', 'score_postop':'mental_score_postop'})

In [21]:
df1 = pd.concat([physical, mental[['mental_score_preop', 'mental_score_postop']]], axis = 1)

In [22]:
df1.drop('index', axis = 1, inplace = True)
df1.drop('nome_modulistica', axis = 1, inplace = True)
columns_names = ['physical_score_preop',
 'physical_score_postop', 
 'mental_score_preop',
 'mental_score_postop',
 'condizioni_meteo',
 'sesso',
 'anni_ricovero',
 'temperatura_media',
 'umidita_media',
 'HI > 27',
 'natural_light']
df1 = df1.reindex(columns=columns_names)
df1.head()

,physical_score_preop,physical_score_postop,mental_score_preop,mental_score_postop,condizioni_meteo,sesso,anni_ricovero,temperatura_media,umidita_media,HI > 27,natural_light
0,27.60,37.40,54.36,58.46,poco nuvoloso,F,70,20.0,69.0,False,True
1,28.15,38.25,57.02,64.62,poco nuvoloso,M,69,18.0,65.0,False,True
2,31.57,55.67,60.27,58.47,poco nuvoloso,M,70,23.0,74.0,False,True
3,34.28,57.08,50.17,54.97,poco nuvoloso,F,66,15.0,72.0,False,True
4,27.43,29.13,54.14,55.74,nebbia al mattino,F,73,18.0,75.0,False,True


## Aggiunta delle differenze pre-post

Genero due nuove colonne contenti le differenze tra gli score post e pre operazione

In [23]:
physical_score_diff = df1['physical_score_postop'] - df1['physical_score_preop']
mental_score_diff = df1['mental_score_postop'] - df1['mental_score_preop']
df1.insert(2,"physical_score_diff", physical_score_diff)
df1.insert(5,"mental_score_diff", mental_score_diff)


Converto le colonne delle variabili categoriche da stringhe a fattori:

In [24]:
df1['condizioni_meteo'] = df1['condizioni_meteo'].astype("category")
df1['sesso'] = df1['sesso'].astype("category")

##Aggiunta differenze physical-mental

In [25]:
physical_mental_diff_postop = df1['physical_score_postop'] - df1['mental_score_postop']
physical_mental_diff_preop = df1['physical_score_preop'] - df1['mental_score_preop']
df1.insert(6,"physical_mental_diff_postop", physical_mental_diff_postop)
df1.insert(7,"physical_mental_diff_preop", physical_mental_diff_preop)
mental_physical_diff_postop = df1['mental_score_postop'] - df1['physical_score_postop'] 
mental_physical_diff_preop = df1['mental_score_preop'] - df1['physical_score_preop']
df1.insert(8,"mental_physical_diff_postop", mental_physical_diff_postop)
df1.insert(9,"mental_physical_diff_preop", mental_physical_diff_preop)

## Splitting del dataset

In [26]:
mental_score_dataset = df[df['nome_modulistica'] == 'SF12 MentalScore']
physical_score_dataset = df[df['nome_modulistica'] == 'SF12 PhysicalScore']

Controllo che il tipo di modulo utilizzato sia sempre SF12 per ogni record

In [27]:
mental_score_dataset['nome_modulistica'].nunique() == physical_score_dataset['nome_modulistica'].nunique() == 1

True

Droppo la colonna nome_modulistica

In [28]:
mental_score_dataset.drop('nome_modulistica', axis = 1, inplace = True)
physical_score_dataset.drop('nome_modulistica',  axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Aggiungo la colonna delle differenze

In [29]:
physical_score_dataset.insert(2,"score_diff", (physical_score_dataset['score_postop'] - physical_score_dataset['score_preop']).round(3))
mental_score_dataset.insert(2,"score_diff", (mental_score_dataset['score_postop'] - mental_score_dataset['score_preop']).round(3))

##Salvataggio dataset

In [30]:
df1.to_csv('dataset_normalizzato_con_differenze.csv', index = False)
mental_score_dataset.to_csv('mental_score_SF12.csv', index = False)
physical_score_dataset.to_csv('physical_score_SF12.csv', index = False)

# Analisi preliminare per la variabile "Natural light"

In [ ]:
df1.groupby('natural_light').size()

In [ ]:
df1['physical_score_postop'].mean()

In [ ]:
df1['mental_score_postop'].mean()

In [ ]:
def f_above_physical(row):
    if row['physical_score_postop'] >= 42.86:
        return 1
    elif row['physical_score_postop'] < 42.86:
        return 0
    
def f_above_mental(row):
    if row['mental_score_postop'] >= 51.63:
        return 1
    elif row['mental_score_postop'] < 51.63:
        return 0

In [ ]:
df1['above_physical'] = df1.apply(f_above_physical, axis = 1)
df1['above_mental'] = df1.apply(f_above_mental, axis = 1)
df1.head()

In [ ]:
df1.groupby(['natural_light', 'above_physical', 'above_mental']).size()

Proporzioni:

Natural_light = False:

quadrante in basso a sinistra (0,0) = 42/121 = 0,347

quadrante in alto a sinistra (0,1) = 32/121 = 0,264

quadrante in basso a destra (1,0) = 15/121 = 0,124

quadrante in alto a destra (1,1) = 32/121 = 0,264


Natural_light = True:

quadrante in basso a sinistra (0,0) = 261/924 = 0,282

quadrante in alto a sinistra (0,1) = 200/924 = 0,216

quadrante in basso a destra (1,0) = 104/924 = 0,113

quadrante in alto a destra (1,1) = 369/924 = 0,399

Per natural_light = True c'è una maggior percentuale di osservazioni in alto a destra (score sia fisico che mentale sopra la media). La luce spinge a dare dei voti più alti?

# Visualizzazione

In [ ]:
#Codice Remo Marconzini
#definisco la griglia
sns.set_style('white')
g = sns.JointGrid(height=10, space=0.001, xlim=(0,100), ylim=(0,100))
g.fig.patch.set_facecolor('#fbf9f4')
g.ax_joint.patch.set_facecolor('#fbf9f4')
g.ax_marg_x.patch.set_facecolor('#fbf9f4')
g.ax_marg_y.patch.set_facecolor('#fbf9f4')

#creare dataframe con le sole righe dove natural_light=True
df1_on = df1[df1['natural_light'] == True]

#creare dataframe con le sole righe dove natural_light=False
df1_off = df1[df1['natural_light'] == False]


# creo vettori per plottare la distribuzione congiunta diversificando per la variabile "natural light"
x, y = df1.physical_score_postop, df1.mental_score_postop
#creo due vettori contenenti in uno le osservazioni con natural_light=True, l'altro con le osservazioni 
# con nantural_light=False
x_off, y_off = df1_off.physical_score_postop, df1_off.mental_score_postop
x_on, y_on = df1_on.physical_score_postop, df1_on.mental_score_postop

#distribuzione congiunta dello score mentale e fisico
sns.kdeplot(x=x, y=y, ax=g.ax_joint,hue=df1["natural_light"], palette = {False:'#7570B3', True:'#E6AB02'}, )
plt.legend(loc='lower right')

#plotto la media congiunta dello score mentale e fisico
mean_m = df1['mental_score_postop'].mean()
mean_p = df1['physical_score_postop'].mean()
g.refline(x=mean_p, y=mean_m, marginal=False)

#distribuzioni marginali light off
sns.kdeplot(x=x_off, ax = g.ax_marg_x, color='#7570B3', linewidth=1.8, fill=True)
sns.kdeplot(y=y_off, ax = g.ax_marg_y, color='#7570B3', linewidth=1.8, fill=True)

#distribuzioni marginali light on
sns.kdeplot(x=x_on, ax = g.ax_marg_x, color='#E6AB02',linewidth=1.8,fill=True)
sns.kdeplot(y=y_on, ax = g.ax_marg_y, color='#E6AB02',linewidth=1.8,fill=True)

#Labels
g.set_axis_labels('SF12 Physical postoperative PROM score 3 months', 'SF12 Mental postoperative PROM score 3 months', fontsize=16)

# Titolo
titolo_italiano = "La somministrazione del questionario in presenza o meno \ndi luce naturale influenza l'indice SF-12 Physical/Mental \nScore postoperatorio?"
titolo_inglese = ""
g.fig.suptitle(
    titolo_italiano,
    x = 0.022,
    y = 1.09,
    ha="left",
    fontsize=20,
    weight="bold",
    wrap = True
)

In [ ]:
#Codice Gianluca Scuri
#creare dataframe con le sole righe dove natural_light=True
df1_on = df1[df1['natural_light'] == True]

#creare dataframe con le sole righe dove natural_light=False
df1_off = df1[df1['natural_light'] == False]

#definisco la griglia
sns.set_style('whitegrid')
g = sns.JointGrid()

#creo i vettorie dei due score per on e off
x_off, y_off = df1_off.physical_score_postop, df1_off.mental_score_postop
x_on, y_on = df1_on.physical_score_postop, df1_on.mental_score_postop

#creo i due kde plot
sns.kdeplot(x=x_on,\
            y=y_on,\
            ax=g.ax_joint,\
            thresh = 0.21,\
            levels = 10,\
            cmap = 'Oranges_d'
            #cmap = None,\
            #colors = shades,\
            #colors = '#E6AB02',\
            #fill = True,\
            )

sns.kdeplot(x=x_off,\
            y=y_off,\
            ax=g.ax_joint,\
            thresh = 0.5,\
            levels = 6,\
            cmap = 'Blues_d'
            )

#distribuzioni marginali light off
sns.kdeplot(x=x_off, ax = g.ax_marg_x, shade = True)
sns.kdeplot(y=y_off, ax = g.ax_marg_y, shade = True)

#distribuzioni marginali light on
sns.kdeplot(x=x_on, ax = g.ax_marg_x, shade = True)
sns.kdeplot(y=y_on, ax = g.ax_marg_y, shade = True)